# Info fields via machine learning

Extract persons from the info fields StartEntryInfo and EndEntryInfo of the [slave registers of Suriname](https://datasets.iisg.amsterdam/dataset.xhtml?persistentId=hdl:10622/CSPBHO) via machine learning

See: https://www.freecodecamp.org/news/getting-started-with-ner-models-using-huggingface/

## 1. Annotating info fields

In [ ]:
import nltk
import pandas as pd
import regex
import transformers

In [ ]:
DATA_FILE = "../../data/suriname/Dataset Suriname Slave and Emancipation Registers Version 1.1.csv"

data = pd.read_csv(DATA_FILE, low_memory=False)

In [ ]:
def add_column_tokens(train):
    train["tokens"] = [ nltk.word_tokenize(text) for text in train["text"] ]
    return train

In [ ]:
def add_column_labels(train):
    train["labels"] = [ len(tokens) * [ "O" ] for tokens in train["tokens"] ]
    return train

In [ ]:
def add_column_numeric_labels(train, numeric_labels):
    train["numeric_labels"] = [ [ numeric_labels[label] for label in labels ] for labels in train["labels"] ]
    return train

In [ ]:
def is_date(day, month, year):
    return regex.search(r"^\d\d\d\d\b", year) and regex.search(r"^\d\d?$", day) and True

In [ ]:
def add_date_tags_to_labels(labels, index):
    labels[index - 2], labels[index - 1], labels[index] = "B-DATE", "I-DATE", "I-DATE"
    return labels

In [ ]:
def label_dates(train):
    for index, row in train.iterrows():
        for i in range(2, len(row["tokens"])):
            if is_date(row["tokens"][i-2], row["tokens"][i-1], row["tokens"][i]):
                add_date_tags_to_labels(row["labels"], i)
    return train       

In [ ]:
def show_annotations(train):
    for index in range(0, len(train)):
        for i in range(0, len(train["labels"][index])):
            print(train["tokens"][index][i], end="")
            if train["labels"][index][i] != "O":
                print("/" + train["labels"][index][i], end="")
            print(" ", end="")
        print("")

In [ ]:
def make_train(data, nbr_of_lines=50):
    train = pd.DataFrame(data["EndEntryInfo"].value_counts()[:nbr_of_lines])
    train = train.rename(columns={"EndEntryInfo": "frequency"})
    train["text"] = train.index
    train["index"] = range(0, len(train))
    train = train.set_index("index")
    return train

In [ ]:
train = make_train(data)
train = add_column_tokens(train)
train = add_column_labels(train)
train = label_dates(train)

In [ ]:
numeric_labels = { "O": 0, "B-DATE": 1, "I-DATE": 2 }

train = add_column_numeric_labels(train, numeric_labels)

### 1.1 Model test

In [ ]:
model_checkpoint = "wietsedv/bert-base-dutch-cased-finetuned-udlassy-ner"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
inputs = tokenizer(train["tokens"][0], is_split_into_words=True)

In [ ]:
inputs

## 2. Tutorial for token classification

https://huggingface.co/docs/transformers/tasks/token_classification

Required modules to install: `pip install transformers datasets evaluate seqeval`

In [ ]:
import datasets

### 2.1 Setting up tutodial data set

In [ ]:
wnut = datasets.load_dataset("wnut_17")

In [ ]:
for key in wnut:
    print(key, len(wnut[key]))

In [ ]:
wnut["train"][0]

In [ ]:
wnut_label_list = wnut["train"].features[f"ner_tags"].feature.names

wnut_label_list

In [ ]:
type(wnut), type(wnut["train"]), type(wnut["train"]["tokens"]), type(label_list)

### 2.2 Setting up info fields data set

In [ ]:
def make_info_field_data(train):
    info_fields_data = {}
    info_fields_data["train"] = datasets.arrow_dataset.Dataset.from_list([ { "id": i, "tokens": train["tokens"][i], "ner_tags": train["labels"][i] } 
                                for i in range(0, int(0.5 + 0.6 * len(train))) ])
    info_fields_data["test"] = datasets.arrow_dataset.Dataset.from_list([ { "id": i, "tokens": train["tokens"][i], "ner_tags": train["labels"][i] } 
                               for i in range(int(0.5 + 0.6 * len(train)), len(train)) ])
    info_fields_data["validation"] = info_fields_data["test"]
    return info_fields_data

In [ ]:
info_fields_data = make_info_field_data(train)

In [ ]:
info_fields_label_list = list(numeric_labels.keys())

### 2.3 Training

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
#training_data = wnut
#label_list = wnut_label_list
training_data = info_fields_data
label_list = info_fields_label_list

In [ ]:
example = training_data["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokenized_input

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_training_data = training_data.map(tokenize_and_align_labels, batched=True)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=13, id2label=id2label, label2id=label2id
)

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False, ### <--- changed
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

### 2.4 Post-training tests

In [ ]:
text = "The Golden State Warriors are an American professional basketball team based in San Francisco."

In [ ]:
# SKIP
from transformers import pipeline

classifier = pipeline("ner", "stevhliu/my_awesome_wnut_model")
classifier(text)

In [ ]:
# SKIP
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("stevhliu/my_awesome_wnut_model")
inputs = tokenizer(text, return_tensors="pt")

In [ ]:
# SKIP
from transformers import AutoModelForTokenClassification
import torch

model = AutoModelForTokenClassification.from_pretrained("stevhliu/my_awesome_wnut_model")
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
import torch

In [ ]:
inputs = tokenizer(text, return_tensors="pt")

In [ ]:
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
predicted_token_class

## 3 Tutorial for IMDB data

https://huggingface.co/transformers/v3.2.0/custom_datasets.html

In [ ]:
from sklearn.model_selection import train_test_split
from pathlib import Path
import torch
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

### 3.1 Read data from disk

In [ ]:
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = [ text_file.read_text() for label_dir in ["pos", "neg"] 
                                    for text_file in (split_dir/label_dir).iterdir() ]
    labels = ( len(list((split_dir/"pos").iterdir())) * [1] + 
               len(list((split_dir/"neg").iterdir())) * [0] )
    return texts, labels

In [ ]:
train_texts, train_labels = read_imdb_split('data/aclImdb/train')
test_texts, test_labels = read_imdb_split('data/aclImdb/test')

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [ ]:
train_texts = train_texts[:1000]
train_labels = train_labels[:1000]
test_texts = test_texts[:1000]
test_labels = test_labels[:1000]
val_texts = val_texts[:1000]
val_labels = val_labels[:1000]

### 3.2 Convert data

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

### 3.3 Fine-tune model with data

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

Error: Kernel Restarting || The kernel for info_fields_ml.ipynb appears to have died. It will restart automatically.

Solutions: 
1. run with smaller datasets (definitely necessary)
2. re-install torchvision: (might not make a difference)
* !pip3 uninstall -y torch torchvision
* !pip3 install torch torchvision

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
results = trainer.predict(val_dataset)

In [ ]:
correct_counter = 0
for i in range(0, len(results[0])):
    result = 1
    if results[0][i][0] > results[0][i][1]:
        result = 0
    if result == val_labels[i]:
        correct_counter += 1
correct_counter

The tutorial contains no example code for testing but see: https://huggingface.co/docs/transformers/main_classes/trainer

## 4. Tutorial for wnut data

https://huggingface.co/transformers/v3.2.0/custom_datasets.html#token-classification-with-w-nut-emerging-entities

In [ ]:
import numpy as np
from pathlib import Path
import regex
from sklearn.model_selection import train_test_split
import torch
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForTokenClassification

### 4.1 Read data from disk

In [ ]:
def read_wnut(file_path):
    file_path = Path(file_path)
    raw_text = file_path.read_text().strip()
    raw_docs = regex.split(r'\n\t?\n', raw_text)
    token_docs = [ [ line.split('\t')[0] for line in doc.split('\n') ] for doc in raw_docs ]
    tag_docs = [ [ line.split('\t')[1] for line in doc.split('\n') ] for doc in raw_docs ]
    return token_docs, tag_docs

In [ ]:
texts, tags = read_wnut('data/wnut17train.conll')

In [ ]:
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

### 4.2 Preprocess data

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc )
tag2id = { tag: id for id, tag in enumerate(unique_tags) }
id2tag = { id: tag for tag, id in tag2id.items() }

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

In [ ]:
train_encodings = tokenizer(train_texts, 
                            is_split_into_words=True, 
                            return_offsets_mapping=True, 
                            padding=True, 
                            truncation=True)
val_encodings =   tokenizer(val_texts, 
                            is_split_into_words=True, 
                            return_offsets_mapping=True, 
                            padding=True, 
                            truncation=True)

In [ ]:
def convert_B_to_I_tag(tag):
    return regex.sub(r"^B", "I", tag)

In [ ]:
def split_tags(tags_in, encodings):
    tags_out = [ [] for _ in range(len(encodings.offset_mapping,)) ]
    for encodings_doc, tags_in_doc, tags_out_doc in zip(encodings.offset_mapping, tags_in, tags_out):
        CLS_seen = False
        SEP_seen = False
        tags_counter = 0
        for encoding in encodings_doc:
            if encoding[1] == 0:
                if not CLS_seen:
                    tags_out_doc.append("CLS")
                    CLS_seen = True
                elif not SEP_seen:
                    tags_out_doc.append("SEP")
                    SEP_seen = True
                else:
                    tags_out_doc.append("PAD")
            elif encoding[0] == 0:
                tags_out_doc.append(tags_in_doc[tags_counter])
                tags_counter += 1
            else:
                tags_out_doc.append(convert_B_to_I_tag(tags_in_doc[tags_counter - 1]))
    return tags_out

In [ ]:
def tags_to_numbers(tags, tag2id):
    return [ [ tag2id[tag] for tag in doc ] for doc in tags ]

In [ ]:
extra_tags = { 'CLS': -100, 'SEP': -100, 'PAD': -100 }

In [ ]:
train_labels = tags_to_numbers( split_tags(train_tags, train_encodings),
                                { **tag2id, **extra_tags})
val_labels =   tags_to_numbers( split_tags(val_tags, val_encodings),
                                { **tag2id, **extra_tags})

In [ ]:
class WNUTDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = WNUTDataset(train_encodings, train_labels)
val_dataset = WNUTDataset(val_encodings, val_labels)

### 4.3 Fine-tune model with data

In [ ]:
model = DistilBertForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(unique_tags))

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
results = trainer.predict(val_dataset)

In [ ]:
def evaluate_results(results):
    correct_count = 0
    missed_count = 0
    wrong_count = 0
    for guesses, corrects in zip(results[0], results[1]):
        for guess_values, correct in zip(guesses, corrects):
            guess = list(guess_values).index(max(guess_values))
            if correct != -100:
                if correct != tag2id['O'] and guess == correct:
                    correct_count += 1
                else:
                    if correct != tag2id['O']:
                        missed_count += 1
                    if guess != tag2id['O']:
                        wrong_count += 1           
    precision = correct_count/(correct_count + wrong_count)
    recall = correct_count/(correct_count + missed_count)
    return precision, recall

In [ ]:
precision, recall = evaluate_results(results)

In [ ]:
precision, recall

In [ ]:
def inspect_results(results, encodings):
    for guess_data, correct_data, token_data in zip(results[0], results[1], encodings):
        for guess_values, correct, token in zip(guess_data, correct_data, tokenizer.convert_ids_to_tokens(token_data)):
            guess = list(guess_values).index(max(guess_values))
            if correct != -100:
                print(token, end="")
                if guess != -100 and guess != tag2id['O']:
                    print("/" + id2tag[guess], end=" ")
                print(" ", end="")
        print()

In [ ]:
inspect_results(results, val_encodings.input_ids)

In [ ]:
tokenizer.convert_ids_to_tokens([101, 137])